In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, BatchNormalization, Dropout

import pandas as pd, numpy as np, tensorflow as tf

In [2]:
def mean_absolute_percentage_error(y_true, y_pred):
    return tf.reduce_mean(tf.abs((y_true-y_pred) / y_true)) * 100.0

In [3]:
#df = pd.read_feather('E:/Skóli/HÍ/Vélaverkfræði Master HÍ/Lokaverkefni/Data/merged-full-25ms-24hr-28-2-24.feather')
df = pd.read_feather('E:/Skóli/HÍ/Vélaverkfræði Master HÍ/Lokaverkefni/Data/merged-w-landscape-full-25ms-24hr-28-2-24.feather')
df = df[df.f < df.fg]
df['gust_factor'] = df.fg / df.f

df = df.drop(['f', 'fg', 'fsdev', 'd', 'dsdev', 'longitude', 'latitude', 'X', 'Y', 'time', 'stod'], axis = 1)# + [f'Landscape_{i}' for i in range(70)], axis = 1)

df_unfolded = df.elevations.apply(pd.Series)
df = pd.concat([df, df_unfolded], axis = 1)

df = df.dropna()
df = df.reset_index(drop = True)


In [4]:
from sklearn.decomposition import PCA

df_landscape_elevation = df.iloc[:, -70:]

df_landscape_elevation = (df_landscape_elevation - df_landscape_elevation.mean()) / df_landscape_elevation.std()

components = 10

pca = PCA(n_components=components)
compressed_features = pca.fit_transform(df_landscape_elevation)

compressed_df = pd.DataFrame(data = compressed_features, columns = ['PC' + str(i) for i in range(components)])

df  = pd.concat([df, compressed_df], axis = 1)

In [5]:
y = df.gust_factor
X = df.drop(['gust_factor'], axis = 1)
X = X.drop(['ws_15', 'ws_250', 'ws_500', 'wd_15', 'wd_250', 'wd_500', 'p_15', 'p_250', 'p_500', 't_15', 't_250', 't_500', 'N_01', 'N_12', 'landscape_points', 'elevations']
           + [i for i in range(70)], axis = 1)

# Changing the type of X,y so as to work with Tensorflow
X, y = X.values.astype(np.float32), y.values.astype(np.float32)
scaler = StandardScaler()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
X_train = scaler.fit_transform(X_train)
X_val = scaler.fit_transform(X_val)
X_test = scaler.fit_transform(X_test)

In [6]:
# Reshape the data for 1D convolution
X_train = np.expand_dims(X_train, axis = -1)
X_val = np.expand_dims(X_val, axis = -1)
X_test = np.expand_dims(X_test, axis = -1)

input_size = X_train.shape[1]

In [9]:
model = tf.keras.Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(input_size, 1)),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),

    Conv1D(32, kernel_size=3, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),

    Flatten(),
    
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),

    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),

    Dense(16, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),

    Dense(1, activation='linear')  # Linear activation for regression
])

In [10]:
model.compile(optimizer = 'adam', loss = mean_absolute_percentage_error)
model.fit(X_train, y_train, epochs = 1000, batch_size = 512)

Epoch 1/1000
26/26 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 110.0477
Epoch 2/1000
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 95.3465
Epoch 3/1000
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 86.1726
Epoch 4/1000
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 77.2641
Epoch 5/1000
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 70.7547
Epoch 6/1000
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 61.1985
Epoch 7/1000
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 53.6970
Epoch 8/1000
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 47.5616
Epoch 9/1000
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 41.0768
Epoch 10/1000
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 35.5607
Epoch 11/1000
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 32.7066
Epoch 12/1000
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 29.5584
Epoch 13/1000
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 26.9946
Epoch 14/1000
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 26.5730
Epoch 15/1000
26/26 ━━━━━━━━

In [11]:
y_pred = model.predict(X_test)

mape = mean_absolute_percentage_error(y_test, y_pred)

print(f"MAPE: {mape}")

115/115 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
MAPE: 8.447904586791992


In [12]:
model.save("./saved_models/cnnWlandscape.keras")